In [1]:
%pip install mwclient


Note: you may need to restart the kernel to use updated packages.


In [2]:
import mwclient 
import time 
site=mwclient.Site("en.wikipedia.org")
page=site.pages["Bitcoin"]

In [3]:
revs=list(page.revisions())

In [4]:
revs[0]

OrderedDict([('revid', 1281323903),
             ('parentid', 1280330283),
             ('minor', ''),
             ('user', 'Davide King'),
             ('timestamp',
              time.struct_time(tm_year=2025, tm_mon=3, tm_mday=19, tm_hour=17, tm_min=46, tm_sec=48, tm_wday=2, tm_yday=78, tm_isdst=-1)),
             ('comment', '/* 2020–present */ copyedit')])

In [5]:
revs=sorted(revs, key=lambda re:["timestamp"])

In [6]:
revs[0]



OrderedDict([('revid', 1281323903),
             ('parentid', 1280330283),
             ('minor', ''),
             ('user', 'Davide King'),
             ('timestamp',
              time.struct_time(tm_year=2025, tm_mon=3, tm_mday=19, tm_hour=17, tm_min=46, tm_sec=48, tm_wday=2, tm_yday=78, tm_isdst=-1)),
             ('comment', '/* 2020–present */ copyedit')])

  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import scipy
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device=0)  # 0 refers to GPU


def find_sentiment(text):
    sent=sentiment_pipeline([text[:250]])[0]
    score=sent["score"]
    if sent["label"]=="NEGATIVE":
        score*= -1
    return score







No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Device set to use cpu


In [11]:
edits={}
for rev in revs:
    date =time.strftime("%y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date]=dict(sentiments=list(),edit_count=0)
    edits[date]["edit_count"] +=1
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))









In [12]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"])>0:
        edits[key]["sentiment"]=mean (edits[key]["sentiments"])
        edits[key]["neg_sentiment"]=len([s for s in edits[key]["sentiments"] if s <0]) /len(edits[key]["sentiments"])

    else:
        edits[key]["sentiment"]=0
        edits[key]["neg_sentiments"]=0


    del edits[key]["sentiments"]








In [13]:
edits

{'25-03-19': {'edit_count': 1,
  'sentiment': -0.9699525833129883,
  'neg_sentiment': 1.0},
 '25-03-13': {'edit_count': 2,
  'sentiment': -0.9965516030788422,
  'neg_sentiment': 1.0},
 '25-03-12': {'edit_count': 1,
  'sentiment': -0.9968662858009338,
  'neg_sentiment': 1.0},
 '25-03-11': {'edit_count': 1,
  'sentiment': -0.857483983039856,
  'neg_sentiment': 1.0},
 '25-03-10': {'edit_count': 1,
  'sentiment': 0.9649892449378967,
  'neg_sentiment': 0.0},
 '25-03-08': {'edit_count': 2,
  'sentiment': -0.9920050501823425,
  'neg_sentiment': 1.0},
 '25-03-03': {'edit_count': 1,
  'sentiment': -0.9947563409805298,
  'neg_sentiment': 1.0},
 '25-02-15': {'edit_count': 1,
  'sentiment': 0.7481209635734558,
  'neg_sentiment': 0.0},
 '25-01-28': {'edit_count': 1,
  'sentiment': 0.5926694273948669,
  'neg_sentiment': 0.0},
 '25-01-27': {'edit_count': 3,
  'sentiment': -0.9918509920438131,
  'neg_sentiment': 1.0},
 '25-01-26': {'edit_count': 4,
  'sentiment': -0.9955549538135529,
  'neg_sentiment'

In [14]:
import pandas as pd 
edits_df=pd.DataFrame.from_dict(edits, orient="index")




In [15]:
edits_df

,edit_count,sentiment,neg_sentiment
25-03-19,1,-0.969953,1.00
25-03-13,2,-0.996552,1.00
25-03-12,1,-0.996866,1.00
25-03-11,1,-0.857484,1.00
25-03-10,1,0.964989,0.00
...,...,...,...
09-10-13,2,-0.227501,0.50
09-08-14,1,0.930021,0.00
09-08-06,2,0.995746,0.00
09-08-05,1,0.748121,0.00


In [16]:
edits_df.index=pd.to_datetime(edits_df.index)

C:\Users\anuka\AppData\Local\Temp\ipykernel_21352\731140064.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  edits_df.index=pd.to_datetime(edits_df.index)


In [17]:
from datetime import datetime

dates=pd.date_range(start="2009-03-08",end=datetime.today())


In [18]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2025-03-16', '2025-03-17', '2025-03-18', '2025-03-19',
               '2025-03-20', '2025-03-21', '2025-03-22', '2025-03-23',
               '2025-03-24', '2025-03-25'],
              dtype='datetime64[ns]', length=5862, freq='D')

In [19]:
edits_df=edits_df.reindex(dates , fill_value=0)

In [20]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,0,0.000000,0.0
2009-03-09,0,0.000000,0.0
2009-03-10,0,0.000000,0.0
2009-03-11,0,0.000000,0.0
2009-03-12,0,0.000000,0.0
...,...,...,...
2025-03-21,1,0.663470,0.0
2025-03-22,0,0.000000,0.0
2025-03-23,0,0.000000,0.0
2025-03-24,1,-0.998504,1.0


In [21]:
rolling_edits=edits_df.rolling(30).mean()

In [22]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-03-08,NaN,NaN,NaN
2009-03-09,NaN,NaN,NaN
2009-03-10,NaN,NaN,NaN
2009-03-11,NaN,NaN,NaN
2009-03-12,NaN,NaN,NaN
...,...,...,...
2025-03-21,1.766667,-0.071803,0.145185
2025-03-22,1.766667,-0.071803,0.145185
2025-03-23,1.766667,-0.071803,0.145185
2025-03-24,1.800000,-0.105087,0.178519


In [23]:
rolling_edits=rolling_edits.dropna()
rolling_edits


,edit_count,sentiment,neg_sentiment
2009-04-06,1.266667,-0.193400,0.233987
2009-04-07,1.266667,-0.193400,0.233987
2009-04-08,1.266667,-0.193400,0.233987
2009-04-09,1.266667,-0.193400,0.233987
2009-04-10,1.266667,-0.193400,0.233987
...,...,...,...
2025-03-21,1.766667,-0.071803,0.145185
2025-03-22,1.766667,-0.071803,0.145185
2025-03-23,1.766667,-0.071803,0.145185
2025-03-24,1.800000,-0.105087,0.178519


In [24]:
rolling_edits.to_csv("Wikipedia_btc_edits.csv")